In [ ]:
%load_ext autoreload

In [ ]:
# Utilities 
from options.options import is_notebook, get_options, update_options, print_options
import argparse
import math
import time
import random
import os
from tqdm.notebook import tqdm, trange
import sys

# Data
import numpy as np
from sklearn.model_selection import train_test_split
from data_utils.ts_dataset import TSDataset
from data_utils.data_utils import prepare_dataloaders
import pandas as pd

from preprocessor.preprocessor import PreprocessingPipeline

# Training Utils
from train_utils.train_utils import *
import wandb

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

# Transformer
from tstransformer.Models import Transformer
from tstransformer.Optim import ScheduledOptim

%autoreload 2

In [ ]:
# Helper function to determin GPU usage
def get_gpu_memory(name = "GPU Memory", verbose = True):
    import nvidia_smi
    nvidia_smi.nvmlInit()

    
    handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
    info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
    
    memory = {"total":  info.total / (1024 * 1024), "free": info.free / (1024 * 1024), "used": info.used / (1024 * 1024)}
    
    if verbose:
        print("#### ", name , "####")
        print("Total memory:", memory["total"], "MB")
        print("Free memory: ", memory["free"], "MB")
        print("Used memory: ", memory["used"], "MB")
        print("")
    
    nvidia_smi.nvmlShutdown()
    
    return memory

#_=get_gpu_memory("Before Empty Cache")
torch.cuda.empty_cache()
_=get_gpu_memory("After Empty Cache")

In [ ]:
opt = get_options()
print_options(opt)

# Data Preprocessing

In [ ]:
# Load the dataset and preprocess it 
aq = pd.read_parquet("/home/yannic/master-thesis/data_air/prsa_data.parquet")

# With Stations
#aq_prep = pd.concat([aq,pd.get_dummies(aq['station'], prefix='station',dummy_na=False)],axis=1).drop(['station'],axis=1).drop(columns=["wind_direction"])

# Without Stations
aq_prepared = aq.drop(['station'],axis=1).drop(columns=["wind_direction"]).iloc[:,4:]

#aq_prepared = aq_prepared.fillna(0)#[:300]

In [ ]:
fill_type = 'ema_fast'           # OPTIONS: 'nan', 'median', 'locf', 'nocb', 'ema', 'ema_fast'
clip_quantile_value = 0.99   # OPTIONS: range(0.0 , 1.0)

preprocessing_pipe = PreprocessingPipeline(fill_type, clip_quantile_value)
aq_preprocessed = preprocessing_pipe.fit_transform(aq_prepared)
aq_preprocessed.shape

In [ ]:
aq_preprocessed = aq_preprocessed.iloc[:10000,:]
print("Nans : ",aq_preprocessed.isnull().sum().sum())
print("Shape: ",aq_preprocessed.shape)
aq_preprocessed.head()

# DataLoader and Transformer

In [ ]:
def validate_epoch(model, validation_dataloader, device, opt):
    model.eval()
    val_loss = 0

    with torch.no_grad():
        for data, target in validation_dataloader:
                
            data = data.to(device)
            target = target.to(device)
            decoder_input = data[:,-1,:].unsqueeze(dim=1)
            
            pred = model(data, decoder_input)
            
            loss = calc_loss(pred, target, opt["loss_func"])
            val_loss += loss.item()
            
    
    return val_loss
    

In [ ]:
def train(model, train_dataloader, validation_datalaoder, optimizer, device, opt):
    
    if opt["use_wandb"]:
        wandb.init(project="masters-thesis", entity="yannicj", config=opt)
        #wandb.config = opt
    
    train_losses = []
    eval_losses = []

    for epoch in range(opt["epoch"]):
        
        # Initiate losses list for epoch
        train_loss = 0
        
        with tqdm(train_dataloader, unit="batch", mininterval=1, leave=True) as tepoch:
            tepoch.set_description(f"Epoch {epoch}")
           
            # Train epoch
            for data, target in train_dataloader:
                tepoch.update(1)
                # Prepare data
                data = data.to(device)
                target = target.to(device)
                decoder_input = data[:,-1,:].unsqueeze(dim=1)
                
                # Trainig data
                optimizer.zero_grad()
                pred = model(data, decoder_input)

                loss = calc_loss(pred, target, opt["loss_func"])
                train_loss += loss.item()

                loss.backward()
                optimizer.step_and_update_lr()
                
                tqdm_loss = "{:10.4f}".format(train_loss)
                tepoch.set_postfix(train_loss=tqdm_loss)
            
            # Validate epoch
            val_loss = validate_epoch(model, validation_datalaoder, device, opt)
            
            # Update Progress Bar
            
            tqdm_train_loss = "{:10.4f}".format(train_loss)
            tqdm_val_losses = "{:10.4f}".format(val_loss)
            
            tepoch.set_postfix({"loss": tqdm_train_loss, "val_loss": tqdm_val_losses})
            tepoch.close()
            
            if opt["use_wandb"]:
                wandb.log({"train_loss": train_loss, "val_loss": val_loss})
                wandb.watch(model)
    
    if opt["use_wandb"]:
        wandb.finish()

In [ ]:
# define the cuda devide
device = torch.device('cuda' if opt["cuda"] else 'cpu')

# Load options
opt = get_options()

# create train, eval, test split
train_split, test_split = train_test_split(aq_preprocessed, test_size=0.15, shuffle=False)
train_split, eval_split = train_test_split(train_split, test_size=0.18, shuffle=False)

# Training dataset, Evaluation dataset in each epoch, Testing dataset after training
train_dataset, train_dataloader = prepare_dataloaders(train_split.values, opt["batch_size"],window_size=opt["window_size"], device=device)
eval_dataset, eval_dataloader = prepare_dataloaders(eval_split.values,  opt["batch_size"],window_size=opt["window_size"], device=device)
test_dataset, test_dataloader = prepare_dataloaders(test_split.values,  opt["batch_size"],window_size=opt["window_size"], device=device)

# Define Transformer
transformer = Transformer(
        n_src_sequence=opt["src_sequence_size"],
        n_trg_sequence=opt["trg_sequence_size"],
        src_pad_idx=opt["src_pad_idx"],
        trg_pad_idx=opt["trg_pad_idx"],
        trg_emb_prj_weight_sharing=opt["proj_share_weight"],
        emb_src_trg_weight_sharing=opt["embs_share_weight"],
        d_k=opt["d_key"],
        d_v=opt["d_value"],
        d_model=opt["d_model"],
        d_sequence_vec=opt["d_sequence"],
        d_inner=opt["d_inner_hidden"],
        n_layers=opt["n_layers"],
        n_head=opt["n_head"],
        dropout=opt["dropout"],
        n_position=opt["d_sequence"],
        scale_emb_or_prj=opt["scale_emb_or_prj"]).to(device)

# Define Optimizer
optimizer = ScheduledOptim(
                optim.Adam(transformer.parameters(), betas=(0.9, 0.98), eps=1e-09),
                opt["lr_mul"], opt["d_model"], opt["n_warmup_steps"])

# print training settings
print_training_settings(device, train_split, eval_split, test_split, opt["loss_func"])

# Start the training process
#train(transformer, train_dataloader, train_dataloader, optimizer, device, opt)

In [ ]:
train(transformer,train_dataloader,eval_dataloader,optimizer,device,opt)

## Make Prediction

In [ ]:

data = data.to(device)
target = target.to(device)
decoder_input = data[:,-1,:].unsqueeze(dim=1)

# Old Functions

In [ ]:
def train_epoch(model, training_data, optimizer, opt, device, smoothing):
    ''' Epoch operation in training phase'''

    model.train()
    total_loss, n_word_total, n_word_correct = 0, 0, 0 

    desc = '  - (Training)   '
    for batch in tqdm(training_data, mininterval=2, desc=desc, leave=False):
        
        
        # prepare data
        src_seq = patch_src(batch.src, opt.src_pad_idx).to(device)
        trg_seq, gold = map(lambda x: x.to(device), patch_trg(batch.trg, opt.trg_pad_idx))

        # forward
        optimizer.zero_grad()
        pred = model(src_seq, trg_seq)

        # backward and update parameters
        loss, n_correct, n_word = cal_performance(
            pred, gold, opt.trg_pad_idx, smoothing=smoothing) 
        loss.backward()
        optimizer.step_and_update_lr()

        # note keeping
        n_word_total += n_word
        n_word_correct += n_correct
        total_loss += loss.item()

    loss_per_word = total_loss/n_word_total
    accuracy = n_word_correct/n_word_total
    return loss_per_word, accuracy

In [ ]:
def eval_epoch(model, validation_data, device, opt):
    ''' Epoch operation in evaluation phase '''

    model.eval()
    total_loss, n_word_total, n_word_correct = 0, 0, 0

    desc = '  - (Validation) '
    with torch.no_grad():
        for batch in tqdm(validation_data, mininterval=2, desc=desc, leave=False):

            # prepare data
            src_seq = patch_src(batch.src, opt.src_pad_idx).to(device)
            trg_seq, gold = map(lambda x: x.to(device), patch_trg(batch.trg, opt.trg_pad_idx))

            # forward
            pred = model(src_seq, trg_seq)
            loss, n_correct, n_word = cal_performance(
                pred, gold, opt.trg_pad_idx, smoothing=False)

            # note keeping
            n_word_total += n_word
            n_word_correct += n_correct
            total_loss += loss.item()

    loss_per_word = total_loss/n_word_total
    accuracy = n_word_correct/n_word_total
    return loss_per_word, accuracy

In [ ]:
def train(model, training_data, validation_data, optimizer, device, opt):
    ''' Start training '''
    
    # Use wandb to plot curves, e.g. perplexity, accuracy, learning rate
    # TODO: Implement this

    log_train_file = os.path.join(opt["output_dir"], 'train.log')
    log_valid_file = os.path.join(opt["output_dir"], 'valid.log')

    print('[Info] Training performance will be written to file: {} and {}'.format(log_train_file, log_valid_file))

    with open(log_train_file, 'w') as log_tf, open(log_valid_file, 'w') as log_vf:
        log_tf.write('epoch,loss,ppl,accuracy\n')
        log_vf.write('epoch,loss,ppl,accuracy\n')

    def print_performances(header, accu, start_time, lr):
        print('  - {header:12} , accuracy: {accu:3.3f} %, lr: {lr:8.5f}, ''elapse: {elapse:3.3f} min'.format(
                  header=f"({header})",accu=100*accu, elapse=(time.time()-start_time)/60, lr=lr))

    #valid_accus = []
    valid_losses = []
    for epoch_i in range(opt["epoch"]):
        print('[ Epoch', epoch_i, ']')

        start = time.time()
        train_loss, train_accu = train_epoch(
            model, training_data, optimizer, opt, device, smoothing=opt["label_smoothing"])
        
        
        # Current learning rate
        lr = optimizer._optimizer.param_groups[0]['lr']
        print_performances('Training', train_accu, start, lr)

        start = time.time()
        valid_loss, valid_accu = eval_epoch(model, validation_data, device, opt)
        valid_ppl = math.exp(min(valid_loss, 100))
        print_performances('Validation', valid_ppl, valid_accu, start, lr)

        valid_losses += [valid_loss]

        checkpoint = {'epoch': epoch_i, 'settings': opt, 'model': model.state_dict()}
        
        if opt["save_mode"] == 'all':
            model_name = 'model_accu_{accu:3.3f}.chkpt'.format(accu=100*valid_accu)
            torch.save(checkpoint, model_name)
        elif opt["save_mode"] == 'best':
            model_name = 'model.chkpt'
            if valid_loss <= min(valid_losses):
                torch.save(checkpoint, os.path.join(opt["output_dir"], model_name))
                print('    - [Info] The checkpoint file has been updated.')

        with open(log_train_file, 'a') as log_tf, open(log_valid_file, 'a') as log_vf:
            log_tf.write('{epoch},{loss: 8.5f},{ppl: 8.5f},{accu:3.3f}\n'.format(
                epoch=epoch_i, loss=train_loss,
                ppl=train_ppl, accu=100*train_accu))
            log_vf.write('{epoch},{loss: 8.5f},{ppl: 8.5f},{accu:3.3f}\n'.format(
                epoch=epoch_i, loss=valid_loss,
                ppl=valid_ppl, accu=100*valid_accu))

        if opt["use_tb"]:
            tb_writer.add_scalars('ppl', {'train': train_ppl, 'val': valid_ppl}, epoch_i)
            tb_writer.add_scalars('accuracy', {'train': train_accu*100, 'val': valid_accu*100}, epoch_i)
            tb_writer.add_scalar('learning_rate', lr, epoch_i)


--------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Testing and Stuff

In [ ]:
train_dataset, train_dataloader = prepare_dataloaders(aq_preprocessed.values, opt["batch_size"],window_size=10)

In [ ]:
i = 1
j = 0
for data, label in train_dataloader:
    x = data[0]
    y = label[0]
    if j==i:
        break
    j+=1

x_df = pd.DataFrame(x.numpy(), columns=['pm25', 'pm10', 'so2', 'no2', 'co', 'o3', 'temperatur', 'pressure',
       'dew_point', 'rain', 'wind_speed'])

y_df = pd.DataFrame(y.numpy(), columns=['pm25', 'pm10', 'so2', 'no2', 'co', 'o3', 'temperatur', 'pressure',
       'dew_point', 'rain', 'wind_speed'])

sample = pd.concat([x_df, y_df]).reset_index().drop(columns=["index"]).round(2).astype(str)
real = aq_preprocessed[i+0:i+11].reset_index().drop(columns=["index"]).round(2).astype(str)

In [ ]:
sample

In [ ]:
real

In [ ]:
(sample == real).any().any()

In [ ]:
pd.DataFrame(train_dataset[3][0].numpy(), columns=['pm25', 'pm10', 'so2', 'no2', 'co', 'o3', 'temperatur', 'pressure',
       'dew_point', 'rain', 'wind_speed']).round(2)